# 2.Marketing_Case-ELVT-Build

## SCIKIT-LEARN

In [50]:
%matplotlib inline

In [51]:
import warnings
warnings.filterwarnings("ignore")

from IPython.display import display

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
import pandas_profiling
import seaborn as sns
import numpy as np
from collections import Counter
import pickle


In [52]:
FILE = 'marketing_training_cleaned.csv'
TEST_DF_FILE = './marketing_test_sklearn.pkl'
MODEL_FILE = './marketing_model.pkl'
NORMALIZATION_DICT = './normalization_dict.pkl' 

# Load dataset
data = pd.read_csv(FILE)

In [53]:
data.head()

,cons.conf.idx,cons.price.idx,contact,day_of_week,default,emp.var.rate,euribor3m,housing,loan,marital,...,nr.employed,poutcome,profession,responded,schooling,campaign1,custAge1,pastEmail1,pcontacted_pdays,pContacted_previous
0,-42.0,93.200,cellular,mon,unknown,-0.1,4.191,no,no,single,...,5195.8,nonexistent,admin.,no,university.degree,1,55.0,no,True,no
1,-42.7,93.918,cellular,mon,no,1.4,4.960,no,no,married,...,5228.1,nonexistent,blue-collar,no,other,1,42.0,no,True,no
2,-36.4,93.994,telephone,mon,no,1.1,4.857,no,no,married,...,5191.0,nonexistent,technician,no,high.school,1,42.0,no,True,no
3,-42.7,93.918,cellular,wed,unknown,1.4,4.962,yes,yes,divorced,...,5228.1,nonexistent,management,no,other,2,55.0,no,True,no
4,-46.2,92.893,cellular,tue,no,-1.8,1.291,yes,no,divorced,...,5099.1,failure,admin.,no,university.degree,5,45.0,yes,True,yes


In [54]:
transformed_variables = [] 

columns = data.columns.copy()
label_name = 'responded'
feature_names = columns.drop(label_name)

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler

normalization_dict = {}

for column in feature_names:
    if data[column].dtype == 'object':
        transformed_variables.append(data[column].name)
        
        lb = LabelBinarizer()
        transformed = lb.fit_transform(data[column].values)
        
        #save binarizer for future
        normalization_dict[column] = lb
        name = data[column].name
        dfOneHot = pd.DataFrame(transformed, 
                                columns =[name+"_"+str(list(set(data[column].values))[i]) for i in range(transformed.shape[1])])
        data = pd.concat([data, dfOneHot], axis=1)
        
    if  data[column].dtype == 'float64':
        transformed_variables.append(data[column].name)
        
        scaler = MinMaxScaler()
        transformed = scaler.fit_transform(data[column].values.reshape(-1,1))
        
        #save binarizer for future
        normalization_dict[column] = scaler
        name = data[column].name
        dfOneHot = pd.DataFrame(transformed, 
                                columns =[name+"_scaled"])
        data = pd.concat([data, dfOneHot], axis=1)
        
#drop transformed variables      
data.drop(transformed_variables, axis=1, inplace=True)

# replace responded and pcontacted_pdays
data.replace(['no',False,'yes',True], [0,0,1,1], inplace=True)

In [55]:
data.head()

,responded,pcontacted_pdays,cons.conf.idx_scaled,cons.price.idx_scaled,contact_telephone,day_of_week_thu,day_of_week_unknown,day_of_week_fri,day_of_week_mon,day_of_week_wed,...,campaign1_6,campaign1_2,campaign1_7+,campaign1_4,campaign1_1,campaign1_5,campaign1_3,custAge1_scaled,pastEmail1_no,pContacted_previous_no
0,0,1,0.368201,0.389322,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0.804348,0,0
1,0,1,0.338912,0.669135,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0.521739,0,0
2,0,1,0.602510,0.698753,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0.521739,0,0
3,0,1,0.338912,0.669135,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0.804348,0,0
4,0,1,0.192469,0.269680,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0.586957,1,1


In [56]:
data.columns

Index(['responded', 'pcontacted_pdays', 'cons.conf.idx_scaled',
       'cons.price.idx_scaled', 'contact_telephone', 'day_of_week_thu',
       'day_of_week_unknown', 'day_of_week_fri', 'day_of_week_mon',
       'day_of_week_wed', 'day_of_week_tue', 'default_unknown',
       'emp.var.rate_scaled', 'euribor3m_scaled', 'housing_unknown',
       'housing_no', 'housing_yes', 'loan_unknown', 'loan_no', 'loan_yes',
       'marital_unknown', 'marital_married', 'marital_single',
       'marital_divorced', 'month_sep', 'month_mar', 'month_aug', 'month_may',
       'month_nov', 'month_dec', 'month_jun', 'month_oct', 'month_jul',
       'month_apr', 'nr.employed_scaled', 'poutcome_success',
       'poutcome_nonexistent', 'poutcome_failure', 'profession_housemaid',
       'profession_unknown', 'profession_student', 'profession_services',
       'profession_technician', 'profession_blue-collar',
       'profession_self-employed', 'profession_retired',
       'profession_entrepreneur', 'profession_ma

## Build and train the model

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Organize our data
label_names = data.responded.name
labels = data[label_name]
feature_names = data.columns.drop(label_names)
features = data[feature_names]


# Look at our data
print(label_names)
print(feature_names)
display(features.head(5))

# Split our data
train, test, train_labels, test_labels = train_test_split(features,
                                                          labels,
                                                          test_size=0.2,
                                                          random_state=25)

# Initialize our classifier
lr = LogisticRegression('l1')

# Train our classifier
model = lr.fit(train, train_labels)


responded
Index(['pcontacted_pdays', 'cons.conf.idx_scaled', 'cons.price.idx_scaled',
       'contact_telephone', 'day_of_week_thu', 'day_of_week_unknown',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_wed',
       'day_of_week_tue', 'default_unknown', 'emp.var.rate_scaled',
       'euribor3m_scaled', 'housing_unknown', 'housing_no', 'housing_yes',
       'loan_unknown', 'loan_no', 'loan_yes', 'marital_unknown',
       'marital_married', 'marital_single', 'marital_divorced', 'month_sep',
       'month_mar', 'month_aug', 'month_may', 'month_nov', 'month_dec',
       'month_jun', 'month_oct', 'month_jul', 'month_apr',
       'nr.employed_scaled', 'poutcome_success', 'poutcome_nonexistent',
       'poutcome_failure', 'profession_housemaid', 'profession_unknown',
       'profession_student', 'profession_services', 'profession_technician',
       'profession_blue-collar', 'profession_self-employed',
       'profession_retired', 'profession_entrepreneur',
       'profession_manag

,pcontacted_pdays,cons.conf.idx_scaled,cons.price.idx_scaled,contact_telephone,day_of_week_thu,day_of_week_unknown,day_of_week_fri,day_of_week_mon,day_of_week_wed,day_of_week_tue,...,campaign1_6,campaign1_2,campaign1_7+,campaign1_4,campaign1_1,campaign1_5,campaign1_3,custAge1_scaled,pastEmail1_no,pContacted_previous_no
0,1,0.368201,0.389322,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0.804348,0,0
1,1,0.338912,0.669135,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0.521739,0,0
2,1,0.602510,0.698753,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0.521739,0,0
3,1,0.338912,0.669135,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0.804348,0,0
4,1,0.192469,0.269680,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0.586957,1,1


In [58]:
#save a TEST portion for future testing
test_df = test.copy()
test_df['response'] = test_labels
test_df.tail(5).to_pickle(TEST_DF_FILE)


## Evaluate model

In [62]:
# Make predictions
preds = lr.predict(test)
print(preds)

# Evaluate accuracy
print(accuracy_score(test_labels, preds))

[0 0 0 ..., 0 0 0]
0.908536585366


## Save Model

In [60]:
#save normalization objects
with open(NORMALIZATION_DICT, 'wb') as handle:
    pickle.dump(normalization_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("Normalization objects saved")

# save the model
with open(MODEL_FILE, 'wb') as handle:
    pickle.dump(gnb, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("Model saved")

Normalization objects saved
Model saved
